In [49]:
# Libraries
import pandas as pd
import numpy as np
import os # Library for interacting with the operating system


# Visualizations
import matplotlib.pyplot as plt

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

## 1. Loading Data

In [50]:
# load dataset
energy_df = pd.read_csv('./dataset/loureiro_energy.csv')  

## 2. Data Preprocessing and Cleaning

In [51]:
# show general information about the dataset
energy_info = energy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46608 entries, 0 to 46607
Columns: 173 entries, Time to Energy_Meter_172
dtypes: float64(172), object(1)
memory usage: 61.5+ MB


In [52]:
# Now select the first 8,000 rows:
energy_df = energy_df.iloc[:8000].reset_index(drop=True)

In [53]:
# Count how many nulls there are in each column
null_counts = energy_df.isnull().sum()

# Filter only columns with up to 100 null values
cols_ate_100 = null_counts[null_counts <= 100]

# To see the result as a Series
print(cols_ate_100)

colunas = cols_ate_100.index.tolist()
print("Colunas com até 100 nulos:", colunas)


Time                  0
Energy_Meter_1        4
Energy_Meter_2       96
Energy_Meter_4        1
Energy_Meter_8       99
Energy_Meter_16      95
Energy_Meter_17       1
Energy_Meter_57      97
Energy_Meter_68      99
Energy_Meter_83      95
Energy_Meter_90       2
Energy_Meter_91       4
Energy_Meter_94      99
Energy_Meter_108     95
Energy_Meter_113     96
Energy_Meter_117     99
Energy_Meter_118     97
Energy_Meter_125      3
Energy_Meter_132      1
Energy_Meter_133     95
Energy_Meter_135     95
Energy_Meter_137     95
Energy_Meter_139    100
Energy_Meter_140      6
Energy_Meter_144     98
Energy_Meter_146     97
Energy_Meter_151    100
Energy_Meter_160     95
Energy_Meter_164     95
dtype: int64
Colunas com até 100 nulos: ['Time', 'Energy_Meter_1', 'Energy_Meter_2', 'Energy_Meter_4', 'Energy_Meter_8', 'Energy_Meter_16', 'Energy_Meter_17', 'Energy_Meter_57', 'Energy_Meter_68', 'Energy_Meter_83', 'Energy_Meter_90', 'Energy_Meter_91', 'Energy_Meter_94', 'Energy_Meter_108', 'Energy_Met

In [54]:
print(f"Dataset contains {energy_df.shape[0]} rows and {energy_df.shape[1]} columns.")
energy_df.head()

Dataset contains 8000 rows and 173 columns.


,Time,Energy_Meter_1,Energy_Meter_2,Energy_Meter_3,Energy_Meter_4,Energy_Meter_5,Energy_Meter_6,Energy_Meter_7,Energy_Meter_8,Energy_Meter_9,...,Energy_Meter_163,Energy_Meter_164,Energy_Meter_165,Energy_Meter_166,Energy_Meter_167,Energy_Meter_168,Energy_Meter_169,Energy_Meter_170,Energy_Meter_171,Energy_Meter_172
0,2022-05-05 12:00:00,0.068,NaN,0.034,0.038,0.000,NaN,NaN,NaN,NaN,...,NaN,0.004,0.0,NaN,NaN,NaN,0.395,NaN,NaN,0.008
1,2022-05-05 12:15:00,0.070,NaN,0.044,0.062,0.006,NaN,NaN,NaN,NaN,...,NaN,0.003,0.0,NaN,NaN,NaN,0.316,NaN,NaN,0.056
2,2022-05-05 12:30:00,0.122,NaN,0.037,0.045,0.000,NaN,NaN,NaN,NaN,...,NaN,0.004,0.0,NaN,NaN,NaN,0.264,NaN,NaN,0.033
3,2022-05-05 12:45:00,0.132,NaN,0.015,0.039,0.000,NaN,NaN,NaN,NaN,...,NaN,0.004,0.0,NaN,NaN,NaN,0.139,NaN,NaN,0.062
4,2022-05-05 13:00:00,0.125,NaN,0.015,0.040,0.032,NaN,NaN,NaN,NaN,...,NaN,0.004,0.0,NaN,NaN,NaN,0.138,NaN,NaN,0.024


In [55]:
# Check unique values for categorical columns in client data
for col in energy_df.select_dtypes(include=["object", "category"]).columns:
    print(f"{col} has {energy_df[col].nunique()} unique values: {energy_df[col].unique()[:5]}", '\n')

Time has 8000 unique values: ['2022-05-05 12:00:00' '2022-05-05 12:15:00' '2022-05-05 12:30:00'
 '2022-05-05 12:45:00' '2022-05-05 13:00:00'] 



---
### 2.1. Handling Missing Data

In [56]:
# check for missing data
print(energy_df.isnull().sum())

Time                   0
Energy_Meter_1         4
Energy_Meter_2        96
Energy_Meter_3      4240
Energy_Meter_4         1
                    ... 
Energy_Meter_168    2298
Energy_Meter_169    2421
Energy_Meter_170    3631
Energy_Meter_171    7420
Energy_Meter_172     573
Length: 173, dtype: int64


In [57]:
# Assuming df is your DataFrame
# List of all meters (e.g. Energy_Meter_1 to Energy_Meter_172)
medidores = [f'Energy_Meter_{i}' for i in range(1, 173)]

# Fill missing values ​​with the average of the 5 closest meters
for medidor in medidores:
    medidores_vizinhos = [m for m in medidores if m != medidor][:5]  # 5 random meters as proxy
    energy_df[medidor] = energy_df[medidor].fillna(energy_df[medidores_vizinhos].mean(axis=1))

### 2.2 Feature Engeneering and Transformation

`Transform date columns to datetime type`

In [58]:
energy_df.columns = energy_df.columns.str.strip()
energy_df['Time']  = pd.to_datetime(energy_df['Time'], errors='coerce')

`Energy Feature Engeneering`

In [59]:
# Identify columns that start with 'Energy_Meter_'
meter_cols = [c for c in energy_df.columns if c.startswith('Energy_Meter_')]

# Adds these columns row by row and stores them in 'Total_Energy_Consumption'
energy_df['Total_Energy_Consumption'] = energy_df[meter_cols].sum(axis=1)


# show the result
print(energy_df['Total_Energy_Consumption'].info())
print(energy_df['Total_Energy_Consumption'].head())

<class 'pandas.core.series.Series'>
RangeIndex: 8000 entries, 0 to 7999
Series name: Total_Energy_Consumption
Non-Null Count  Dtype  
--------------  -----  
8000 non-null   float64
dtypes: float64(1)
memory usage: 62.6 KB
None
0     9.036
1     9.613
2    10.160
3    10.143
4     9.850
Name: Total_Energy_Consumption, dtype: float64


/var/folders/xj/vmtsvzgs6zj1l876xt4q0nkw0000gn/T/ipykernel_43241/1798902470.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  energy_df['Total_Energy_Consumption'] = energy_df[meter_cols].sum(axis=1)


In [60]:
# Saving cleaned dataset
directory_name = 'dataset'
output_filename = 'energy_data_cleaned.csv'

# Build the full path
output_path = os.path.join(directory_name, output_filename)

# Save the DataFrame to a CSV file
total_energy.to_csv(output_path, index=False)